In [26]:
import os
import json
import codecs
import locale
import requests
import datetime

import tabula

import pandas as pd


In [27]:
#----------------------------------------------------------------
# Configurations section
#----------------------------------------------------------------

# Url of the pdf file to download:
url_region_pdf = "http://www.salute.gov.it/imgs/C_17_notizie_4370_0_file.pdf"

#----------------------------------------------------------------
#
#----------------------------------------------------------------
now = datetime.datetime.now()
sample_date = now.strftime("%d/%m/%Y")

locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')
ok_statuses = [200, 201, 202]
data_file_path = os.path.join("..","data")
#data_file_path = os.path.join(os.sep,"tmp")

pdf_file_name = os.path.join(os.sep, "tmp", "temp_data_file.pdf")
it_data_file = os.path.join(data_file_path, "virus-it.csv")
it_tmp_data_file = os.path.join(data_file_path, "virus-it-{dt}.csv".format(dt=now.strftime("%Y%m%d")))


In [28]:
#result = requests.get(url_province_latest)
#if result.status_code not in ok_statuses:
#    assert "Get data failed. Received error code: {er}".format(er=str(result.status_code))
    
#with codecs.open(prov_data_file, "w", "utf-8") as fh:
#    fh.write(result.text)


In [29]:
#----------------------------------------------------------------
#
#----------------------------------------------------------------

def get_web_file(url):
    """
    
    :param url: 
    :return (rv, content):
    """
    rv = False
    try:
        result = requests.get(url_region_pdf)
        if result.status_code not in ok_statuses:
            print("Get data failed. Received error code: {er}".format(er=str(result.status_code)))
    except Exception as ex:
        print("get_web_file failed - {ex}".format(ex=ex))
    else:
        rv = True
    return (rv, result.content)    
        
def save_content_to_file(file_name, content):
    """
    
    :param file_name: 
    :return rv:
    """
    rv = False
    try:
        with open(file_name, "wb") as fh:
            fh.write(content)
    except Exception as ex:
        print("save_content_to_file failed - {ex}".format(ex=ex))
    else:
        rv = True
    return rv
   
def pdf_to_dataframe(pdf_file_name):
    """
    
    :param pdf_file_name: 
    :return rv:
    """
    rv = False
    df = None
    try:
        df = tabula.read_pdf(pdf_file_name)
        #print("Number of dataframes read: {n}.".format(n=len(df)))
    except Exception as ex:
        print("pdf_to_dataframe failed - {ex}".format(ex=ex))
    else:
        rv = True
    return (rv, df)

def get_row_from_Italy_df(df):
    """
    :param df: 
    :return (rv, content):
    """
    rv = False
    csv_row = []
    try:
        csv_row.append(sample_date)
        
        infected_label = df.columns.values[0]
        infected_num = locale.atoi(df.columns.values[1])
        csv_row.append(str(infected_num))
        
        df.set_index(["ATTUALMENTE POSITIVI"], inplace=True)
        df["Totals"] = df[df.columns[0]].apply(lambda row: int(row) if row.is_integer() else int(row * 1000) )
        
        tot_deads = df.loc["TOTALE DECEDUTI", ['Totals']]
        tot_deads = str(int(tot_deads.values[0]))
        csv_row.append(tot_deads)
        
        tot_recovered = df.loc["TOTALE GUARITI", ['Totals']]
        tot_recovered = str(int(tot_recovered.values[0]))
        csv_row.append(tot_recovered)
        
        tot_infected = df.loc["CASI TOTALI", ['Totals']]
        tot_infected = str(int(tot_infected.values[0]))
        csv_row.append(tot_infected)
        
        csv_row = ",".join(csv_row)
        rv = True
        
    except Exception as ex:
        print("get_row_from_Italy_df failed - {ex}".format(ex=ex))
    else:
        rv = True
    return (rv, csv_row)

def append_row(file_name, row):
    """
    
    :param file_name:
    :param row:
    """
    rv = False
    try:
        with open(file_name, 'a') as fh:
            row = row + "\n"
            fh.write(row)
    except Exception as ex:
        print("append_row failed - {ex}".format(ex=ex))
    else:
        rv = True
    return rv

In [30]:


df_regions = None
df_Italy = None

rv = False
result = get_web_file(url_region_pdf)
if result[0] == True:
    rv = save_content_to_file(pdf_file_name, result[1])

if rv == True:
    df_list = pdf_to_dataframe(pdf_file_name)
if rv == True:
    df_regions = df_list[1][0]
    df_Italy = df_list[1][1]
else:
    assert False, "Unable to download or save the data file."

ita_row = get_row_from_Italy_df(df_Italy)    
if ita_row[0] == True:
    with open(it_data_file, 'r') as src, open(it_tmp_data_file, 'w') as dst: 
            dst.write(src.read())
            dst.close()
            src.close()
    rv = append_row(it_tmp_data_file, ita_row[1])
    

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Apr 01, 2020 6:01:11 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F1 are not implemented in PDFBox and will be ignored
Apr 01, 2020 6:01:11 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F2 are not implemented in PDFBox and will be ignored



In [31]:
it_tmp_data_file

'../data/virus-it-20200401.csv'